# 图像视觉分析 - 使用Qwen3-VL

本notebook使用Qwen3-VL模型对图像进行专业的视觉分析，包括：
- **构图分析**：画面布局、视觉引导、黄金分割等
- **光影分析**：光源方向、明暗对比、光影层次
- **色彩分析**：色调、饱和度、色彩搭配、情绪表达

## 1. 环境设置

In [ ]:
# 安装必要的库
!pip install transformers torch pillow numpy matplotlib requests qwen-vl-utils -q

In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Markdown
import requests
from io import BytesIO

## 2. 加载Qwen3-VL模型

In [ ]:
# 模型配置
model_name = "Qwen/Qwen2-VL-7B-Instruct"  # 可以根据需要选择 2B 或 7B 版本

# 加载模型和处理器
print("加载模型中...")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

processor = AutoProcessor.from_pretrained(model_name)
print("模型加载完成！")

## 3. 定义分析函数

In [ ]:
def load_image(image_path):
    """
    加载图像，支持本地路径和URL
    """
    if image_path.startswith('http://') or image_path.startswith('https://'):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(image_path)
    return image.convert('RGB')


def analyze_image(image_path, analysis_type="comprehensive"):
    """
    使用Qwen3-VL分析图像
    
    参数:
        image_path: 图像路径（本地或URL）
        analysis_type: 分析类型
            - "composition": 构图分析
            - "lighting": 光影分析
            - "color": 色彩分析
            - "comprehensive": 综合分析（默认）
    """
    
    # 定义不同类型的分析提示词
    prompts = {
        "composition": """请详细分析这张图像的构图，从以下维度进行专业评估：

### 1. 景别分析
- 景别类型：特写(Close-up)、近景(Medium Close-up)、中景(Medium Shot)、全景(Full Shot)、远景(Long Shot)、大远景(Extreme Long Shot)
- 景别选择的合理性与表现力

### 2. 取景范围与框架构成
- 人物入镜情况：单人、双人、多人、群像
- 特殊视角：Over-the-Shoulder(过肩镜头)、POV(主观视角)、Dutch Angle(荷兰角)
- 取景框架是否完整

### 3. 主体位置与视觉重心
- 主体在画面中的位置(中心、偏左、偏右、三分线位置等)
- 视觉重心是否明确
- 是否符合构图法则(三分法、黄金分割、对称、中心构图等)

### 4. 画面纯净度
- 是否存在干扰元素(杂乱背景物、不必要的前景遮挡、穿帮物体)
- 构图是否简洁干净
- 背景虚化程度(浅景深/深景深)
- 背景与主体的分离度

### 5. 场景层次
- 前景、中景、背景的层次关系
- 是否有前景遮挡(有意为之的景深营造 vs 无意的遮挡)
- 空间纵深感

### 6. 线条与几何结构
- 垂直线是否歪斜(建筑物、柱子等是否倾斜)
- 水平线是否倾斜(地平线、桌面等)
- 对角线构图的运用
- 引导线(Leading Lines)的存在与作用
- S曲线或Z型构图
- 建筑透视是否正常(有无桶形/枕形畸变，透视是否合理)

### 7. 视觉节奏
- 画面中的重复元素
- 节奏感与韵律感
- 视觉流动性

### 8. 负空间(Negative Space)
- 留白的运用
- 负空间是否合理
- 呼吸感与压迫感

### 9. 构图评价
- 整体构图的优点
- 存在的问题
- 改进建议""",
        
        "lighting": """请详细分析这张图像的光影：
1. 光源特征：主光源位置、光源类型（自然光/人造光）、光源数量
2. 明暗对比：高光、中间调、阴影的分布和比例
3. 光影层次：光影过渡是否柔和，是否有明显的明暗分界线
4. 氛围营造：光影如何影响画面情绪和氛围
5. 光影优缺点及改进建议""",
        
        "color": """请详细分析这张图像的色彩：
1. 色调：主要色调、冷暖倾向
2. 色彩搭配：配色方案（互补色、类似色、对比色等）
3. 饱和度：色彩的鲜艳程度和纯度
4. 明度：整体明度分布，高调/低调
5. 情绪表达：色彩传达的情感和氛围
6. 色彩优缺点及改进建议""",
        
        "comprehensive": """请从专业摄影和影视制作的角度，全面分析这张图像：

## 一、构图分析

### 1. 景别与取景
- 景别类型(特写/近景/中景/全景/远景)
- 人物入镜情况(单人/双人/多人)
- 特殊视角(Over-the-Shoulder/POV等)

### 2. 主体位置与构图法则
- 主体位置与视觉重心
- 构图法则运用(三分法、黄金分割、对称等)

### 3. 画面质量
- 干扰元素与画面纯净度
- 背景虚化程度
- 场景层次(前景/中景/背景)
- 前景遮挡情况

### 4. 线条与几何
- 垂直线/水平线是否倾斜
- 引导线、对角线、S曲线
- 建筑透视是否正常

### 5. 视觉节奏与负空间
- 视觉节奏与流动性
- 负空间的运用与呼吸感

## 二、光影分析
1. 光源特征与光线方向
2. 明暗对比与光影层次
3. 光影营造的氛围

## 三、色彩分析
1. 色调与色彩搭配
2. 饱和度与明度
3. 色彩的情绪表达

## 四、综合评价
1. 整体视觉效果
2. 优点与特色
3. 可改进之处及具体建议"""
    }
    
    # 获取对应的提示词
    prompt = prompts.get(analysis_type, prompts["comprehensive"])
    
    # 构建消息
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": image_path,
                },
                {"type": "text", "text": prompt},
            ],
        }
    ]
    
    # 处理输入
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)
    
    # 生成分析结果
    print("分析中...")
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=2048,
            temperature=0.7,
            top_p=0.9,
        )
    
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    
    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )[0]
    
    return output_text


def display_analysis(image_path, analysis_result):
    """
    展示图像和分析结果
    """
    # 加载并显示图像
    image = load_image(image_path)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title('分析图像', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
    
    # 显示分析结果
    print("\n" + "="*80)
    print("分析结果")
    print("="*80 + "\n")
    display(Markdown(analysis_result))

## 4. 使用示例

### 4.1 综合分析示例

In [ ]:
# 设置图像路径（可以是本地路径或URL）
image_path = "your_image.jpg"  # 请替换为你的图像路径

# 进行综合分析
result = analyze_image(image_path, analysis_type="comprehensive")
display_analysis(image_path, result)

### 4.2 单独进行构图分析

In [ ]:
# 构图分析
composition_result = analyze_image(image_path, analysis_type="composition")
display_analysis(image_path, composition_result)

### 4.3 单独进行光影分析

In [ ]:
# 光影分析
lighting_result = analyze_image(image_path, analysis_type="lighting")
display_analysis(image_path, lighting_result)

### 4.4 单独进行色彩分析

In [ ]:
# 色彩分析
color_result = analyze_image(image_path, analysis_type="color")
display_analysis(image_path, color_result)

## 5. 批量分析多张图像

In [ ]:
# 批量分析示例
image_paths = [
    "image1.jpg",
    "image2.jpg",
    "image3.jpg",
    # 添加更多图像路径
]

for i, img_path in enumerate(image_paths, 1):
    print(f"\n{'='*80}")
    print(f"分析第 {i} 张图像: {img_path}")
    print(f"{'='*80}\n")
    
    try:
        result = analyze_image(img_path, analysis_type="comprehensive")
        display_analysis(img_path, result)
    except Exception as e:
        print(f"处理 {img_path} 时出错: {str(e)}")
    
    print("\n" + "="*80 + "\n")

## 6. 使用在线图像示例

In [ ]:
# 使用URL图像进行分析
online_image_url = "https://example.com/image.jpg"  # 替换为实际的图像URL

result = analyze_image(online_image_url, analysis_type="comprehensive")
display_analysis(online_image_url, result)

## 7. 自定义分析提示词

In [ ]:
def custom_analyze(image_path, custom_prompt):
    """
    使用自定义提示词进行分析
    """
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": custom_prompt},
            ],
        }
    ]
    
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(model.device)
    
    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=2048)
    
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    
    output_text = processor.batch_decode(
        generated_ids_trimmed,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )[0]
    
    return output_text


# 使用自定义提示词
custom_prompt = """请分析这张图像的电影感：
1. 画面是否具有电影质感
2. 色彩调性（电影级调色）
3. 景深和虚化效果
4. 情绪氛围的营造
5. 与经典电影画面的相似之处"""

result = custom_analyze(image_path, custom_prompt)
display_analysis(image_path, result)

## 8. 注意事项

1. **模型选择**：可以选择 `Qwen2-VL-2B-Instruct` 或 `Qwen2-VL-7B-Instruct`，7B模型效果更好但需要更多显存
2. **图像格式**：支持常见图像格式（JPG, PNG等）
3. **图像大小**：建议图像不要过大，可以适当调整大小以加快处理速度
4. **GPU内存**：确保有足够的GPU内存，7B模型约需16GB显存
5. **提示词优化**：可以根据具体需求调整提示词以获得更精准的分析结果